In [2]:
import pandas as pd
import numpy as np
from itertools import product

In [3]:
start_month = "2023-08-01"

end_month = "2023-08-31"

dates = pd.date_range(
    start_month,
    end_month
)

country_list = [
    "KZ",
    "PT",
    "FR",
    "BR",
    "GR"
]

df_users_columns = {
    "customer_id": str,
    "registration_date": str,
    "first_deposit_date": str,
    "country": "category",
    "traffic_type": "category",
    "partner_id": str,
    "cost_usd": float
}

df_deposits_columns = {
    "payment_id": str,
    "created": str,
    "customer_id": str,
    "deposit_usd": float
}

df_withdrawals_columns = {
    "payment_id": str,
    "created": str,
    "customer_id": str,
    "withdrawal_usd": float
}

In [4]:
def read_users() -> pd.DataFrame:
    df_users = pd.read_csv(
        "users.csv",
        usecols = df_users_columns.keys(),
        dtype = df_users_columns,
        sep = ",",
        parse_dates = [
            "registration_date",
            "first_deposit_date"
        ]
    )

    df_users["partner_id"] = df_users["partner_id"].apply(
        lambda x: x.split(".")[0] if isinstance(x, str) else x
    )
    df_users["partner_id"] = df_users["partner_id"]\
    .astype("category")

    return df_users


def create_spine(
        dates: list,
        country_list: list
) -> pd.DataFrame:
    data = product(country_list, dates)

    df_spine = pd.DataFrame(
        data,
        columns = [
            "country",
            "date"
        ]
    )

    return df_spine


def find_registration_dynamic_by_country(
        df_users: pd.DataFrame,
        df_spine: pd.DataFrame
) -> pd.DataFrame:
    df_users_grouped = df_users.groupby(
        [
            "country",
            "registration_date"
        ],
        as_index = False
    ).size()\
    .rename(
        columns = {
            "size": "new_registration_count",
            "registration_date": "date"
        }
    )

    df_spine = df_spine.merge(
        df_users_grouped,
        on = [
            "country",
            "date"
        ],
        how = "left"
    )

    df_spine["new_registration_count"] = df_spine["new_registration_count"].fillna(0.0)

    df_spine.to_csv(
        "registration_dynamic_by_country.csv",
        sep = ",",
        index = False
    )

    return df_spine


def find_first_deposits_dynamic_by_country(
        df_users: pd.DataFrame,
        df_spine: pd.DataFrame
) -> pd.DataFrame:
    df_users_grouped = df_users[
        df_users["first_deposit_date"].between(
            start_month,
            end_month
        )
    ].groupby(
        [
            "country",
            "first_deposit_date"
        ],
        as_index = False
    ).size()\
    .rename(
        columns = {
            "first_deposit_date": "date",
            "size": "first_deposit_count"
        }
    )

    df_spine = df_spine.merge(
        df_users_grouped,
        on = [
            "country",
            "date"
        ],
        how = "left"
    )

    df_spine["first_deposit_count"] = df_spine["first_deposit_count"].fillna(0.0)

    df_spine.to_csv(
        "first_deposits_dynamic_by_country.csv",
        index = False,
        sep = ","
    )
    
    return df_spine


def find_conversion_from_regist_to_deposit(
        df_users: pd.DataFrame,
        df_spine: pd.DataFrame
) -> pd.DataFrame:
    df_queryed_by_regist_date = df_users.groupby(
        [
            "registration_date",
            "country"
        ],
        as_index = False
    ).size()\
    .rename(
        columns = {
            "size": "new_customers_count",
            "registration_date": "date"
        }
    )

    df_queryed_by_regist_date_with_deposit = df_users[
        df_users["first_deposit_date"].between(
            start_month,
            end_month
        )    
    ].groupby(
        [
            "registration_date",
            "country"
        ],
        as_index = False,
    ).size()\
    .rename(
        columns = {
            "size": "new_customers_with_deposit_count",
            "registration_date": "date"
        }
    )

    df_spine = df_spine.merge(
        df_queryed_by_regist_date,
        on = [
            "date",
            "country"
        ],
        how = "left"
    ).merge(
        df_queryed_by_regist_date_with_deposit,
        on = [
            "date",
            "country"
        ],
        how = "left"
    )

    col_to_fill = [
        "new_customers_count",
        "new_customers_with_deposit_count"
    ]
    df_spine[col_to_fill] = df_spine[col_to_fill].fillna(0.0)
    

    df_spine["conversion, %"] = df_spine["new_customers_with_deposit_count"]\
    .div(df_spine["new_customers_count"])\
    .mul(100)\
    .round(3)

    df_spine.to_csv(
        "conversion_from_regist_to_deposit.csv",
        index = False,
        sep = ";"
    )

    return df_spine


def find_same_date_deposit(
        df_users: pd.DataFrame
) -> pd.DataFrame:
    df_group_total: pd.DataFrame = df_users.groupby(
        [
            "country"
        ],
        as_index = False
    ).size()\
    .rename(
        columns = {
            "size": "new_customers_total"
        }
    )
    
    df_group_total_same_date: pd.DataFrame = df_users[
        df_users["registration_date"].eq(
            df_users["first_deposit_date"]
        )
    ].groupby(
        [
            "country"
        ],
        as_index = False
    ).size()\
    .rename(
        columns = {
            "size": "new_customers_total_same_date"
        }
    )
    
    df_total = df_group_total.merge(
        df_group_total_same_date,
        on = [
            "country"
        ],
        how = "left"
    )

    df_total["same_date_deposit, %"] = df_total["new_customers_total_same_date"]\
    .div(df_total["new_customers_total"])\
    .mul(100)\
    .round(3)

    df_total.to_csv(
        "same_date_deposit.csv",
        index = False,
        sep = ";"
    )
    
    return df_total


def read_deposits() -> pd.DataFrame:
    df_deposits = pd.read_csv(
        "deposits.csv",
        usecols = df_deposits_columns.keys(),
        dtype = df_deposits_columns,
        sep = ",",
        parse_dates = [
            "created"
        ]
    )

    return df_deposits


def process_and_combine_df_users_and_df_deposits(
        df_users: pd.DataFrame,
        df_deposits: pd.DataFrame
) -> pd.DataFrame:
    df_users = df_users[
        df_users["first_deposit_date"].between(
            start_month,
            end_month
        )
    ]

    df_combined = df_users.merge(
        df_deposits,
        on = [
            "customer_id"
        ],
        how = "left"
    ).sort_values(
        by = [
            "customer_id",
            "created"
        ]
    )

    df_combined["deposit_number"] = df_combined.groupby(
        [
            "customer_id"
        ],
        as_index = False
    ).head(7)\
    .groupby(
        [
            "customer_id"
        ],
        as_index = False
    )["customer_id"]\
    .rank(
        method = "first",
        ascending = True
    )

    df_combined: pd.DataFrame = df_combined.groupby(
        [
            "country",
            "deposit_number"
        ],
        as_index = False
    ).agg(
        **{
            "total_deposited, usd": (
                "deposit_usd", "sum"
            )
        }
    )

    max_by_country = df_combined.groupby(
        [
            "country"
        ],
        as_index = False
    )["total_deposited, usd"]\
    .max()\
    .rename(
        columns = {
            "total_deposited, usd": "total_deposited, usd MAX"
        }
    )

    df_combined = df_combined.merge(
        max_by_country,
        on = [
            "country"
        ],
        how = "left"
    )

    df_combined["%"] = df_combined["total_deposited, usd"]\
    .div(df_combined["total_deposited, usd MAX"])\
    .mul(100)\
    .round(3)

    df_combined["deposit_number"] = df_combined["deposit_number"]\
    .astype(int)\
    .map("{} deposit".format)

    df_combined.to_csv(
        "funnel_area.csv",
        sep = ";",
        index = False
    )

    return df_combined


def read_withdrawals() -> pd.DataFrame:
    df_withdrawals = pd.read_csv(
        "withdrawals.csv",
        usecols = df_withdrawals_columns.keys(),
        dtype = df_withdrawals_columns,
        sep = ",",
        parse_dates = [
            "created"
        ]
    )

    return df_withdrawals


def prepare_dfs_for_six_task(
        df_users: pd.DataFrame,
        df_deposits: pd.DataFrame,
        df_withdrawals: pd.DataFrame
) -> tuple[
    pd.DataFrame,
    pd.DataFrame, 
    pd.DataFrame
]:
    select_columns = [
        "customer_id",
        "country",
        "partner_id",
        "cost_usd"
    ]
    df_queryed = df_users[
        (
            df_users["first_deposit_date"].between(
                start_month,
                end_month
            )
        )
        &
        (
            df_users["traffic_type"] == "paid"
        )
    ][select_columns]

    df_users_deposits = df_queryed.merge(
        df_deposits,
        on = [
            "customer_id"
        ],
        how = "left"
    )

    df_users_deposits = df_users_deposits[
        df_users_deposits["created"].dt.month.between(
            8,
            10
        )
    ]

    df_users_withdrawals = df_queryed.merge(
        df_withdrawals,
        on = [
            "customer_id"
        ],
        how = "left"
    )

    df_users_withdrawals = df_users_withdrawals[
        df_users_withdrawals["created"].dt.month.between(
            8,
            10
        )
    ]

    df_unique_customers = df_queryed[
        [
            "customer_id",
            "country",
            "partner_id",
            "cost_usd"
        ]
    ].drop_duplicates()\
    .reset_index(
        drop = True
    )

    return df_users_deposits, df_users_withdrawals, df_unique_customers


def find_deposit_and_withdrawals(
        df_unique_customers: pd.DataFrame,
        df_users_deposits: pd.DataFrame,
        df_users_withdrawals: pd.DataFrame
) -> pd.DataFrame:
    
    df_users_deposits["month_number"] = df_users_deposits["created"].dt.month
    df_users_withdrawals["month_number"] = df_users_withdrawals["created"].dt.month
    
    df_users_deposits_grouped: pd.DataFrame = df_users_deposits.groupby(
        [
            "customer_id",
            "month_number"
        ],
        as_index = False
    )["deposit_usd"]\
    .sum()\
    .pivot(
        index = "customer_id",
        columns = "month_number",
        values = "deposit_usd"
    ).add_suffix(
        "_month_deposits"
    )

    df_users_withdrawals_grouped = df_users_withdrawals.groupby(
        [
            "customer_id",
            "month_number"
        ],
        as_index = False
    )["withdrawal_usd"]\
    .sum()\
    .pivot(
        index = "customer_id",
        columns = "month_number",
        values = "withdrawal_usd"
    ).add_suffix(
        "_month_withdrawal"
    )


    df_combined = df_unique_customers.merge(
        df_users_deposits_grouped,
        on = [
            "customer_id"
        ],
        how = "left"
    ).merge(
        df_users_withdrawals_grouped,
        on = [
            "customer_id"
        ],
        how = "left"
    )
    
    columns = df_combined.columns[-6:]
    df_combined[columns] = df_combined[columns].fillna(0.0)

    return df_combined


def find_hold(
        df_total: pd.DataFrame
) -> pd.DataFrame:
    df_total["hold_8_month"] = df_total["8_month_deposits"]\
    .sub(df_total["8_month_withdrawal"])\
    .round(4)

    df_total["hold_9_month"] = df_total["9_month_deposits"]\
    .sub(df_total["9_month_withdrawal"])\
    .round(4)

    df_total["hold_10_month"] = df_total["10_month_deposits"]\
    .sub(df_total["10_month_withdrawal"])\
    .round(4)

    columns = [
        "customer_id",
        "country",
        "partner_id",
        "cost_usd",
        "hold_8_month",
        "hold_9_month",
        "hold_10_month"
    ]
    df_total = df_total[columns]

    df_total["total_hold"] = df_total[
        df_total.columns[-3:]
    ].sum(axis=1)\
    .round(3)

    with pd.ExcelWriter(
        "customer_holds.xlsx"
    ) as writter:
        df_total.to_excel(
            writter,
            index = False,
            sheet_name = "customers"
        )
        writter.sheets["customers"].set_column(
            0,
            len(df_total.columns) - 1,
            20
        )

    return df_total


def find_ROI_by_country_and_partner_id(
        df_result: pd.DataFrame
) -> pd.DataFrame:
    df_result_grouped: pd.DataFrame = df_result.groupby(
        [
            "partner_id",
            "country"
        ],
        as_index = False
    ).agg(
        **{
            "total_costs": (
                "cost_usd", "sum"
            ),
            "total_holds": (
                "total_hold", "sum"
            )
        }
    )

    df_result_grouped["ROI"] = df_result_grouped["total_holds"]\
    .div(df_result_grouped["total_costs"])\
    .round(4)

    with pd.ExcelWriter(
        "ROI by partner_id and country.xlsx"
    ) as writter:
        df_result_grouped.to_excel(
            writter,
            index = False,
            sheet_name = "ROI"
        )
        writter.sheets["ROI"].set_column(
            0,
            len(df_result_grouped.columns) - 1,
            25
        )

    return df_result_grouped

In [5]:
df_users = read_users()
df_deposits = read_deposits()
df_withdrawals = read_withdrawals()
df_spine = create_spine(
    dates,
    country_list
)

df_users.head()

,customer_id,registration_date,first_deposit_date,country,traffic_type,partner_id,cost_usd
0,500000,2023-08-01,NaT,GR,paid,177,NaN
1,500001,2023-08-01,NaT,BR,paid,143,NaN
2,500002,2023-08-01,NaT,GR,organic,NaN,NaN
3,500003,2023-08-01,NaT,BR,paid,143,NaN
4,500004,2023-08-01,NaT,PT,organic,NaN,NaN


# 1

In [6]:
df_registration_dynamic_by_country = find_registration_dynamic_by_country(
    df_users,
    df_spine
)

C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_users_grouped = df_users.groupby(


# 2

In [7]:
df_first_deposits_dynamic_by_country =  find_first_deposits_dynamic_by_country(
    df_users,
    df_spine
)

C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:86: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ].groupby(


# 3

In [8]:
df_conversion_from_regist_to_deposit = find_conversion_from_regist_to_deposit(
    df_users,
    df_spine
)

C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:124: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_queryed_by_regist_date = df_users.groupby(
C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:143: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ].groupby(


# 4

In [9]:
df_same_date_deposit = find_same_date_deposit(df_users)

C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:197: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_group_total: pd.DataFrame = df_users.groupby(
C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:213: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ].groupby(


# 5

In [10]:
df_combined = process_and_combine_df_users_and_df_deposits(
    df_users,
    df_deposits
)

C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:302: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_combined: pd.DataFrame = df_combined.groupby(
C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:316: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_by_country = df_combined.groupby(


# 6

In [11]:
df_users_deposits, df_users_withdrawals, df_unique_customers = prepare_dfs_for_six_task(
    df_users,
    df_deposits,
    df_withdrawals
)

df_total = find_deposit_and_withdrawals(
    df_unique_customers,
    df_users_deposits,
    df_users_withdrawals
)

df_result = find_hold(df_total)
df_result = find_ROI_by_country_and_partner_id(df_result)

C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total["total_hold"] = df_total[
C:\Users\Jean_Sky_Poor\AppData\Local\Temp\ipykernel_21608\4131266613.py:555: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_result_grouped: pd.DataFrame = df_result.groupby(
